### Creating and Persisting an ML Model

In [91]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/student-mat.csv', sep=';')
df['qual_student'] = np.where(df['G3']>=15, 1, 0)
X = df.drop(['G3', 'qual_student'], axis=1)
y = df['qual_student']

In [115]:
X['school'] = X['school'].replace(to_replace=['GP','MS'],value=[0,1])
X['sex'] = X['sex'].replace(to_replace=['F','M'],value=[0,1])
X['address'] = X['address'].replace(to_replace=['U','R'],value=[0,1])
X['famsize'] = X['famsize'].replace(to_replace=['LE3','GT3'],value=[0,1])
X['Pstatus'] = X['Pstatus'].replace(to_replace=['T','A'],value=[0,1])
X['Mjob'] = X['Mjob'].replace(to_replace=['teacher','health','services','at_home','other'],value=[0,1,2,3,4])
X['Fjob'] = X['Fjob'].replace(to_replace=['teacher','health','services','at_home','other'],value=[0,1,2,3,4])
X['reason'] = X['reason'].replace(to_replace=['home','reputation','course','other'],value=[0,1,2,3])
X['guardian'] = X['guardian'].replace(to_replace=['mother','father','other'],value=[0,1,2])
X['schoolsup'] = X['schoolsup'].replace(to_replace=['yes','no'],value=[0,1])
X['famsup'] = X['famsup'].replace(to_replace=['yes','no'],value=[0,1])
X['paid'] = X['paid'].replace(to_replace=['yes','no'],value=[0,1])
X['activities'] = X['activities'].replace(to_replace=['yes','no'],value=[0,1])
X['nursery'] = X['nursery'].replace(to_replace=['yes','no'],value=[0,1])
X['higher'] = X['higher'].replace(to_replace=['yes','no'],value=[0,1])
X['internet'] = X['internet'].replace(to_replace=['yes','no'],value=[0,1])
X['romantic'] = X['romantic'].replace(to_replace=['yes','no'],value=[0,1])

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [133]:
from sklearn.feature_selection import SelectKBest, f_regression
fs = SelectKBest(score_func=f_regression, k='all')
fs.fit(X_train, y_train)
X_train_fs = fs.transform(X_train)
weights = {}
for i in range(len(fs.scores_)):
	weights[X_train.columns[i]] = fs.scores_[i]
top_features = list(dict(sorted(weights.items(), key=lambda item: item[1], reverse=True)).keys())[:5]
top_features

['G1', 'G2', 'Medu', 'Mjob', 'failures']

In [134]:
X_train = X_train[top_features]
X_test = X_test[top_features]

Import scikit-learn and build a random forest classifer

In [148]:
from sklearn.ensemble import RandomForestClassifier as rf
import sklearn
clf = rf()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [149]:
pred = clf.predict(X_test)
print(sklearn.metrics.f1_score(y_test, pred, average='binary'))
print(sklearn.metrics.precision_score(y_test, pred, average='binary'))
print(sklearn.metrics.recall_score(y_test, pred, average='binary'))
print(sklearn.metrics.accuracy_score(y_test, pred))

0.9743589743589743
0.95
1.0
0.9915966386554622


In [163]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, './dockerfile/apps/model.pkl')

['./dockerfile/apps/model.pkl']